In [1]:
import pandas as pd
import time
import datetime
import logging
import re
import numpy as np

from pathlib import Path
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

In [2]:
logging.basicConfig(filename='lol_data.log',  
                    level=logging.DEBUG)

In [3]:
#function that gets the html of all solo queue matches from league of Graphs of a given summoner
def lographs_htmlScrapper(summoner):
        button_css = 'button[class="see_more_ajax_button"]'
        driver = webdriver.Edge('C:\Git Projects\Ironhack/edgedriver_win64\msedgedriver.exe')
        driver.get("https://www.leagueofgraphs.com/summoner/lan/" + summoner)
        
        menu = driver.find_element_by_css_selector('a[data-dropdown="drop-queueTypes-recentgames"]')
        menu_item = driver.find_element_by_css_selector('a[data-additional-queue-id="420"]')
        ActionChains(driver).move_to_element(menu).pause(0.5).click(menu).pause(0.5).click(menu_item).perform()        #select option from menu
                    #while button exists get more matches 
    
        logging.debug("Getting data from the page..")
        while(True):  
            time.sleep(np.random.randint(4,5))
            try:
                button = driver.find_element_by_css_selector(button_css)                           
                ActionChains(driver).pause(3).move_to_element(button).pause(0.5).click(button).perform()

            except Exception as ex:
                
                print(f"{summoner},1 Exception found {ex}")
                break
        page_source = driver.page_source
        try:
            rank = driver.find_element_by_css_selector('div[class="leagueTier"]').text
            rank = re.sub(r' \d+ LP','', rank)
        except Exception as ex:
            rank = 'Unranked'
            print(f"{summoner},2 Exception found {ex}")
            
        driver.close()
        return page_source, rank

In [4]:
#function that takes a list of summoner names and creates a list of dataframes for each summoner in the list
def create_dataframe(summoners):
    match_history = []
    logging.debug("Creating a dataframe for each summoner...")
    for summoner in summoners:
            html, rank = lographs_htmlScrapper(summoner) #getting the pagesource of the player
            matches = pd.read_html(html)[0] #there are many tables in the page, we only want the first
            matches = matches.iloc[:,[2,4]] #take the columns we need
            matches.insert(0, 'Rank' , str(rank), True)
            matches.insert(0,'Summoner Name', str(summoner), True)
            matches.columns = ['Summoner Name','Rank','Result','Score']
            
            match_history.append(matches)
            
    return match_history

In [5]:
#the function create_dataframe only create a df without separating the info, we do that here
def df_cleaning(df):
    results = [item.lstrip() for item in df['Result'] if type(item) == str and item != 'See more']
    score = [item.lstrip() for item in df['Score'] if type(item) == str and item != 'See more']
    status = []
    date = []
    duration= []
    kda =[]
    cs = []
    k = []
    d=[]
    a=[]
    
    s=[]
    
    print(f"Cleaning and  creating columns for {df['Summoner Name'].iloc[0]} data...")
    logging.debug(f"Cleaning and  creating columns for {df['Summoner Name'].iloc[0]} data...")
    
    
    for result,score in zip(results,score):
        print(result, score)
        if result == 'See more'or score == 'See more':
            
            break
        
        if (dt := re.findall(r'\d+ days ago',result)):
            dt = datetime.datetime.now().date() - datetime.timedelta(days=int(re.findall(r'\d+',dt[0])[0]))
            date.append(dt)
            
        elif (dt := re.findall(r'a day ago',result)):
            dt = datetime.datetime.now().date() - datetime.timedelta(days = 1)
            date.append(dt)
    
        else:
            dt = datetime.datetime.now().date()
            date.append(dt)
            
              
        status.append(re.findall(r'^(Defeat|Victory|Remake)',result)[0])
        duration.append(int(re.findall(r'\d+',re.findall(r'\d+min',result)[0])[0]))           
        s.append(re.findall(r'\d+\s+/\s+\d+\s+/\s+\d+',score)[0])
        cs.append(int(re.findall(r'\d+',re.findall(r'\d+ CS',score)[0])[0]))                   
   

    for item in s:
        temp = re.findall(r'\d+',item)
        x=int(temp[0])
        y=int(temp[1])
        z=int(temp[2])
        k.append(x)
        d.append(y)
        a.append(z)
        if y ==0:        
            kda.append(round(((x+z)/1),2))
        else:
            kda.append(round(((x+z)/y),2))
    
    new_df = pd.DataFrame(list(zip(status,date,duration,k,d,a,kda,cs))) 
    new_df.insert(0,'Rank', df['Rank'], True)
    new_df.insert(0,'Summoner Name', df['Summoner Name'], True)
    new_df.columns = ["Summoner Name",'Rank',"W/L", "Date", "Duration (min)" ,"K" ,"D","A","KDA", "CS"]
    new_df.drop(new_df[new_df['W/L'] == "Remake"].index, inplace = True)
    new_df = new_df.reset_index(drop=True)
    return new_df

In [6]:
#function that takes a list of dataframes, and a list of names and saves each data frame with the names
def save_dataframes(dataframes, names):
    for dataframe, name in zip(dataframes, names):
        if ' ' in name:
            name = re.sub(' ','_',name)           
        dataframe.to_csv(f'match_histories/{name}_soloq_match_history.csv', index = False)

In [7]:
def get_matchistories(summoners):
    match_history = create_dataframe(summoners)
    clean_match_history = []

    for match in match_history: #cleaning dataframes
        m = df_cleaning(match)
        clean_match_history.append(m)
     
    try: #saving those dataframes
        save_dataframes(clean_match_history, summoners)
        logging.info('Dataframes saved!')
    except:
        logging.exception('An Exception ocurred:')

In [25]:
def create_csv_allmatches():

    df = pd.DataFrame()
    folder = Path('match_histories').rglob('*.csv')
    files = [x for x in folder]

    for file in files:
        print(file)
        f = pd.read_csv(file)  
        print(f)
        df =df.append(f).reset_index(drop=True)
        
    df.to_csv(f'match_histories/all_soloq_match_history.csv', index = False)

# And it all comes down to this

In [26]:
Iron = ['CHAPLI', 'Robertelproxd','TITO MATX' , 'farcom', 'JeffersonSlayerX', 'Rheagars' , 'albertsoto','Grygory RM',  'Wilcekn VI', 'xciberwolfx']#Hierro
Bronze = ['TiburonsinMujaja', 'Pastel de limon','kronno timer', 'xxjhin hierro4' , 'Hugo Roberto', 'EyF123', 'alex0906', 'xEveuSx', 'Gatrix2103', 'perlanegra40']#Bronce
Silver = ['xJoe98', 'Rekkles Support', 'NightmarePhoenix', 'letal endry', 'dracakplay1', 'rodrigo 5793', 'maxxamX', 'samuelioo','M1staMcFly', 'PanchitaPalantir'] #Plata
Gold = ['SweetBlood', 'Saeekz' , 'SkullToxic' , 'ninna' , 'TheKiller0704', 'The Try End', 'Fayta',  'RS Aldebaran', 'FerMx', 'MrKokoBunji']#Oro
Platinum =['Medussyn', 'RazerZero', 'TheStrikeOut', 'EI Trapo', 'Trash Bandicot', 'Kinako', '6thSithLord', 'IGothboi', 'Aleicrag', 'Light Gd Me']#PLatino
Diamond = ['AJDeus', 'Sariel548', 'Jar Jar Pink', 'el osito felpudo' , 'ÆvttasihdoÆ' ,'Fieldset', 'Taco de Azúcar', 'ColmilloGamerDLB', 'roflcow6', 'lards']#Diamante
Master = ['Reíya Edvil', 'Only Fidd', 'ABC XYZ', 'Rogers Sama', 'Kensh1', 'Wınter', 'ACS LuckyStar', 'El Mega', 'TAB Promesse', 'Mr Pan']#Maestro
GrandMaster =['Spykies', 'Kiwi', 'GodSungJinWoo', 'Worst Smite LAN', 'Eudus', 'Carillon', 'Ratoncito 777', 'Masamune Hongo', 'SPX Cesaroxx', 'Tw Mrzebrapug055'] #Gran Maestro
Challenger = ['El Brayayin', 'Twitch Bejjaniii', 'Lixiu', 'Jaunyyy', 'bro', 'Kazuo Tadashi', 'Jarabee','bro','TigoBolt Putin', 'Soy Benet'] #Challenger 
Friends = ['Lopso', ' EI Ritardo','WeIinton Quiw', 'Cosa', 'T O X I C B O Y', 'DiaCraOne', 'Elisma97']

In [27]:
#recibe una lista
get_matchistories(Iron)
get_matchistories(Bronze)
get_matchistories(Silver)
get_matchistories(Gold)
get_matchistories(Platinum)
get_matchistories(Diamond)
get_matchistories(Master)
get_matchistories(GrandMaster)
get_matchistories(Challenger)
get_matchistories(Friends)

create_csv_allmatches()

match_histories\6thSithLord_soloq_match_history.csv
   Summoner Name        Rank      W/L        Date  Duration (min)   K   D   A  \
0    6thSithLord  Platinum I   Defeat  2020-03-12              34   5   5   9   
1    6thSithLord  Platinum I  Victory  2020-03-05              27   5   6   7   
2    6thSithLord  Platinum I  Victory  2020-03-05              25  10   2   5   
3    6thSithLord  Platinum I   Defeat  2020-03-05              26   6   6   2   
4    6thSithLord  Platinum I   Defeat  2020-03-05              21   2   4   4   
..           ...         ...      ...         ...             ...  ..  ..  ..   
91   6thSithLord  Platinum I   Defeat  2020-01-18              29   9   6   5   
92   6thSithLord  Platinum I   Defeat  2020-01-18              28   5  10  12   
93   6thSithLord  Platinum I   Defeat  2020-01-17              29   3   6  10   
94   6thSithLord  Platinum I  Victory  2020-01-17              35  12   7  16   
95   6thSithLord  Platinum I   Defeat  2020-01-17        

match_histories\Kazuo_Tadashi_soloq_match_history.csv
     Summoner Name        Rank      W/L        Date  Duration (min)   K  D  \
0    Kazuo Tadashi  Challenger  Victory  2020-11-01              15   6  0   
1    Kazuo Tadashi  Challenger  Victory  2020-11-01              26   0  1   
2    Kazuo Tadashi  Challenger   Defeat  2020-11-01              28   7  5   
3    Kazuo Tadashi  Challenger   Defeat  2020-11-01              16   2  3   
4    Kazuo Tadashi  Challenger  Victory  2020-11-01              26  16  4   
..             ...         ...      ...         ...             ...  .. ..   
97   Kazuo Tadashi  Challenger   Defeat  2020-09-09              28   5  7   
98   Kazuo Tadashi  Challenger   Defeat  2020-09-09              34   7  7   
99   Kazuo Tadashi  Challenger  Victory  2020-09-07              30  15  5   
100  Kazuo Tadashi  Challenger  Victory  2020-09-07              30  12  5   
101  Kazuo Tadashi  Challenger  Victory  2020-08-29              23   6  5   

      A  

match_histories\Rogers_Sama_soloq_match_history.csv
    Summoner Name         Rank      W/L        Date  Duration (min)   K  D  \
0     Rogers Sama  GrandMaster  Victory  2020-12-12              21  10  6   
1     Rogers Sama  GrandMaster  Victory  2020-12-04              25  10  4   
2     Rogers Sama  GrandMaster  Victory  2020-12-04              21   2  0   
3     Rogers Sama  GrandMaster  Victory  2020-12-04              28  12  7   
4     Rogers Sama  GrandMaster   Defeat  2020-11-30              20   2  6   
..            ...          ...      ...         ...             ...  .. ..   
98    Rogers Sama  GrandMaster   Defeat  2020-09-18              24   3  7   
99    Rogers Sama  GrandMaster  Victory  2020-09-18              27   3  4   
100   Rogers Sama  GrandMaster  Victory  2020-09-18              27  12  4   
101   Rogers Sama  GrandMaster  Victory  2020-09-18              33   8  6   
102   Rogers Sama  GrandMaster   Defeat  2020-09-18              19   8  4   

      A   K